In [1]:
import pandas as pd
import numpy as np 
from sklearn.neighbors import NearestNeighbors

In [2]:
#creating a dataframe of movies_metadata
file1 = 'data/movies.csv'
movies = pd.read_csv(file1)


In [3]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [4]:
#I just need some columns - id, original_title, original_language, vote_count 

movies_redux= movies[['id', 'original_title', 'original_language', 'vote_count']]

In [5]:
movies_redux.isnull().sum() # I don't have any null value

id                   0
original_title       0
original_language    0
vote_count           0
dtype: int64

In [6]:
#I just want en movies
movies_redux_en = movies_redux[(movies_redux['original_language'] == 'en') & (movies_redux['vote_count'] > 999)]
movies_redux_en.shape

(912, 4)

In [8]:
file2 = 'data/ratings.csv'
rating = pd.read_csv(file2)
rating.shape

(26024289, 4)

In [9]:
#I just want user when they have more then 999 votes 
number_votes = rating['userId'].value_counts() > 999
index = number_votes[number_votes].index
rating_number_votes = rating[rating['userId'].isin(index)]
rating_number_votes

,userId,movieId,rating,timestamp
17291,229,1,3.0,1037826148
17292,229,2,3.0,1037827385
17293,229,4,2.0,1037830942
17294,229,5,1.0,1037136141
17295,229,7,2.0,1037828170
...,...,...,...,...
26023517,270887,171439,1.0,1500925554
26023518,270887,171755,3.0,1493850948
26023519,270887,173149,5.0,1497402632
26023520,270887,173405,5.0,1496285130


In [10]:
#  I don't have any null value
rating_number_votes.drop(['timestamp'], axis=1, inplace=True)
rating_number_votes.head()

C:\Users\Vinicius\AppData\Local\Temp\ipykernel_6136\2555312359.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rating_number_votes.drop(['timestamp'], axis=1, inplace=True)


,userId,movieId,rating
17291,229,1,3.0
17292,229,2,3.0
17293,229,4,2.0
17294,229,5,1.0
17295,229,7,2.0


In [11]:
#Doing merge for join 2 datasets 

merge = rating_number_votes.merge(movies_redux_en, left_on='movieId', right_on='id')

In [12]:
merge.drop(['id', 'movieId', 'vote_count','original_language'], axis=1, inplace=True)
merge.shape

(174919, 3)

In [13]:
#We need to pivot the table
merge_pivot = merge.pivot_table(columns='userId', index='original_title', values='rating')
merge_pivot.shape

(412, 2509)

In [14]:
merge_pivot.fillna(0, inplace=True)

In [15]:
#Now we can use start our model
movie_sparse = csr_matrix(merge_pivot)

NameError: name 'csr_matrix' is not defined

In [ ]:
model = NearestNeighbors(algorithm='brute')
model.fit(movie_sparse)

NearestNeighbors(algorithm='brute')

In [ ]:
#Now you can choice your filme and see recommendation
#You just need to choice one filme in the list. I didn't make treatment for autofill or like(IN SQL for example)
movie_input = str(input('What is your film ? '))
distance, sugestions = model.kneighbors(merge_pivot.filter(items=[movie_input], axis=0).values.reshape(1,-1))

for c in range(len(sugestions)):
    print(merge_pivot.index[sugestions[c]])

Index(['Top Gun', 'Black Hawk Down', 'Meet the Fockers', 'Brokeback Mountain',
       'A.I. Artificial Intelligence'],
      dtype='object', name='original_title')
